In [1]:
import os

In [2]:
os.chdir("../")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class CoverLetterConfig:
    artifacts_file: Path
    save_cover_letter_dir: Path
    save_job_description_dir: Path
    cover_letter_rules:Path
    qualifications:Path
    job_description:Path
    model: str
    cover_letter_prompt_templates: dict
    job_description_prompt_templates:dict


In [57]:
from langgraph.graph import END, StateGraph
from typing_extensions import TypedDict
@dataclass(frozen=True)
class CoverLetterState(TypedDict):
   
    job_description : str
    job_description_summary: dict
    cover_letter_draft : str
    human_reviewer: str
    cover_letter_feedback : str
    human_feedback : str
    edited_cover_letter : str
    num_steps : int

In [54]:
from dotenv import load_dotenv
from pathlib import Path


from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser

from job_aplication_agent.utils.common import  load_json,read_yaml,create_directories
from job_aplication_agent import logger
from job_aplication_agent.constants import *
load_dotenv()

True

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        models_filepath = MODELS_FILE_PATH,
        prompt_template = PROMPT_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.models = read_yaml(models_filepath)
        self.prompt_template = read_yaml(prompt_template)
    
        

    
    def get_cover_letter_config(self) -> CoverLetterConfig:
        """
        
        Returns:
            
        """
        
        config = self.config
        cover_letter_config = self.config.cover_letter
        prompt_template = self.prompt_template
        create_directories([config.root_dir])
        
        create_cover_letter_config = CoverLetterConfig(
            artifacts_file=  config.aritfacts_root,
            save_cover_letter_dir= cover_letter_config.save_cover_letter_dir,
            save_job_description_dir= cover_letter_config.save_job_description_dir,
            cover_letter_rules= cover_letter_config.cover_letter_rules,
            qualifications= cover_letter_config.qualifications,
            job_description= cover_letter_config.job_description,
            model= self.models.Llama3,
            cover_letter_prompt_templates= prompt_template.cover_letter_prompt_templates,
            job_description_prompt_templates= prompt_template.job_description_prompt_templates
        ) 

        return create_cover_letter_config

In [22]:
def load_txt(file: Path) -> str:
    with open(file, 'r', encoding='utf-8') as f:
        # Read the entire content of the file
        content = f.read()
    return content

In [61]:
class GetCoverLetter:
    def __init__(self, config:CoverLetterConfig):
        self.config = config
        self.cover_letter_prompt_templates = self.config.cover_letter_prompt_templates
        self.job_description_prompt_templates = config.job_description_prompt_templates

       
        try:
            self.qualifications = load_txt(Path(self.config.qualifications))
            logger.info("Qualifications loaded")
        except Exception as e:
            logger.error("Loading qualification error: {e}")
        try:
            self.rules = load_txt(Path(self.config.cover_letter_rules))
            logger.info("Cover letter rules loaded")
        except Exception as e:
            logger.error("Loading cover letter rules error: {e}")

    def create_cover_letter_agents(self):
        llm = ChatGroq(temperature=0,model_name = self.config.model)
        logger.info(f"Llm initialized")
        

        #Agents
        try:
            draft_cover_letter_prompt = PromptTemplate(
                template=self.cover_letter_prompt_templates.draft_cover_letter_template,
                input_variables=["JOB_DESCRIPTION", "QUALIFICATIONS"]
            )
            self.draft_cover_letter_agent = draft_cover_letter_prompt | llm | StrOutputParser()
            logger.info("Draft Cover Letter Agent created")
        except Exception as e:
            logger.error(f"Creating Draft Cover Letter Agent error: {e}")
        
        try:
            cover_letter_feedback_prompt = PromptTemplate(
                template=self.cover_letter_prompt_templates.cover_letter_feedback_template,
                input_variables=["RULES","COVER_LETTER","JOB_DESCRIPTION","QUALIFICATIONS"])
            self.cover_letter_feedback_agent = cover_letter_feedback_prompt | llm | StrOutputParser()
            logger.info(f"Cover Letter Feedback Agent created")
        except Exception as e:
            logger.error(f"Creating Cover Letter Feedback Agent error {e}")

        try:
            cover_letter_editor_prompt = PromptTemplate(
                template=self.cover_letter_prompt_templates.cover_letter_editor_template,
                input_variables=["FEEDBACK","COVER_LETTER","QUALIFICATIONS"])
            self.cover_letter_editor_agent = cover_letter_editor_prompt | llm | StrOutputParser()
            logger.info(f"Cover Letter Editor Agent created")
        except Exception as e:
            logger.error(f"Creating Cover Letter Editor Agent error {e}")


       

        
    

  
    def get_cover_letter_draft(self,state):
        logger.info(f"Creating cover letter draft")
        job_description_summary = state["job_description_summary"]
        num_steps = int(state['num_steps'])
        num_steps += 1
        try:
            cover_letter_draft = self.draft_cover_letter_agent.invoke({
                "JOB_DESCRIPTION": job_description_summary,
                "QUALIFICATIONS": str(self.qualifications)
            })
            logger.info(f"Cover letter draft created")
        except Exception as e:
            logger.error(f"Creating cover letter draft error {e}")

        return {"cover_letter_draft": cover_letter_draft, "num_steps":num_steps}

    def get_cover_letter_feedback(self,state):
        logger.info(f"Getting feedback from cover letter draft")
        cover_letter_draft = state["cover_letter_draft"]
        job_description_summary = state["job_description_summary"]
        num_steps = int(state['num_steps'])
        num_steps += 1

        try:
            cover_letter_feedback= self.cover_letter_feedback_agent.invoke({"RULES":str(self.rules),
                                                                            "COVER_LETTER":cover_letter_draft,
                                                                            "JOB_DESCRIPTION":job_description_summary,
                                                                        "QUALIFICATIONS":str(self.qualifications)})
            logger.info(f"Feedback for cover letter draft created")
        except Exception as e:
            logger.error(f"Creating feedback for cover letter draft error {e}")
            
        return {"cover_letter_feedback": cover_letter_feedback, "num_steps":num_steps}

    def get_edited_cover_letter(self,state):
        
        logger.info(f"Editing cover letter draft based on feedback")
        cover_letter_draft = state["cover_letter_draft"]
        cover_letter_feedback = state["cover_letter_feedback"]
        human_feedback = state["human_feedback"]
        num_steps = int(state['num_steps'])
        num_steps += 1

        if human_feedback != "proceed with agent feedback":
            cover_letter_feedback = cover_letter_feedback + "\n" + human_feedback
        
        edited_cover_letter= self.cover_letter_editor_agent.invoke({"FEEDBACK":cover_letter_feedback,
                                                                    "COVER_LETTER":cover_letter_draft,
                                                                    "QUALIFICATIONS":str(self.qualifications)})
        return {"edited_cover_letter": edited_cover_letter, "num_steps":num_steps}


    def get_human_feedback(self,state):
        """
       

        Args:
            state (dict): Current state containing agent feedback.

        Returns:
            dict: Updated state with human feedback.
        """
   
        feedback = state['cover_letter_feedback']
        num_steps = int(state['num_steps'])
        num_steps += 1


        print(f"Feedback: \n {feedback}")
        print("Add aditional feedback if necesary")
        human_feedback = input()
        
        return {"human_feedback": human_feedback}
            
        
        
    def get_human_reviewer(self,state):
        print(f"Are you satisfied with the cover letter")
        human_review = input()
        edited_cover_letter = state["edited_cover_letter"]
        if edited_cover_letter:
            return {"cover_letter_draft":edited_cover_letter, "human_review": human_review}
        else:
            return {"human_review": human_review}
               
    def route_to_edit(self, state):
        """
        
        Args:
           

        Returns:
            
           
        """
     
       
        if state["human_review"] == "yes":

            logger.info(f"Finished cover letter creation.")
            return "no_edit"
        
        else:
            logger.info(f"Cover letter needs changes getting feedback.")
            return "edit"
        

    def state_printer(self,state):
        """
        Prints the current state of the processing.

        Args:
            state (dict): The current state of the processing.
        """
        
        logger.info("---STATE PRINTER---")
        logger.info(f"Final cover letter: {state['edited_cover_letter']} \n")
        logger.info(f"Num Steps: {state['num_steps']} \n")



    def create_graph_agents(self, workflow):
        """
       
        """
        
        #nodes
        workflow.add_node("get_job_description_summary", self.get_job_description_summary) 
        workflow.add_node("get_cover_letter_draft", self.get_cover_letter_draft)
        workflow.add_node("get_human_reviewer", self.get_human_reviewer)
        workflow.add_node("get_cover_letter_feedback", self.get_cover_letter_feedback)
        workflow.add_node("get_human_feedback", self.get_human_feedback)
        workflow.add_node("get_edited_cover_letter", self.get_edited_cover_letter)
        workflow.add_node("state_printer", self.state_printer)

        #edges
        workflow.set_entry_point("get_job_description_summary")
        workflow.add_edge("get_job_description_summary","get_cover_letter_draft")
        workflow.add_edge("get_cover_letter_draft","get_human_reviewer")
        workflow.add_conditional_edges(
            "get_human_reviewer",
            self.route_to_edit,
            {
                "no_edit": "state_printer",
                "edit": "get_cover_letter_feedback",
            },
        )
        workflow.add_edge("get_cover_letter_feedback", "get_human_feedback")
        workflow.add_edge("get_human_feedback", "get_edited_cover_letter")
        workflow.add_edge("get_edited_cover_letter", "get_human_reviewer")
        workflow.add_edge("state_printer", END)

        #compile
        try:
            app = workflow.compile()
            logger.info("Agent graph created")
        except Exception as e:
            logger.error(f"Creation of agent graph error: {e}")
             
        return  app
    
    def run_cover_letter_agents(self,app):
        """
      

        Args:
           
        Returns:
         
        """
        try:
            job_description = load_txt(Path(self.config.job_description))
            logger.info("Job description loaded")
        except Exception as e:
            logger.error("Loading job description error: {e}")
        
        inputs = {"job_description":job_description,"num_steps":0}
        try:
            logger.info("Initialazing agent workflow")
            output = app.invoke(inputs)
            logger.info("Finalizing agent workflow")
        except Exception as e:
            logger.error(f"Agent workflow failed : {e}")
        
        
        
        return output 

In [62]:
config = ConfigurationManager()
cover_letter_config = config.get_cover_letter_config()
get_cover_letter= GetCoverLetter(cover_letter_config)
get_cover_letter.create_cover_letter_agents()
app = get_cover_letter.create_graph_agents(StateGraph(CoverLetterState))



# job_description_summary = get_cover_letter.get_job_description_summary()
# cover_letter_draft = get_cover_letter.get_cover_letter_draft(job_description_summary)
# cover_letter_feedback = get_cover_letter.get_cover_letter_feedback(cover_letter_draft,job_description_summary)
# final_cover_letter = get_cover_letter.get_edited_cover_letter(cover_letter_feedback,cover_letter_draft)

[2024-06-22 15:51:11,462: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-06-22 15:51:11,464: INFO: common: yaml file: models.yaml loaded successfully:]
[2024-06-22 15:51:11,470: INFO: common: yaml file: prompt_template.yaml loaded successfully:]
[2024-06-22 15:51:11,473: INFO: 3165304408: Qualifications loaded:]
[2024-06-22 15:51:11,474: INFO: 3165304408: Cover letter rules loaded:]
[2024-06-22 15:51:12,038: INFO: 3165304408: Llm initialized:]
[2024-06-22 15:51:12,040: INFO: 3165304408: Draft Cover Letter Agent created:]
[2024-06-22 15:51:12,041: INFO: 3165304408: Cover Letter Feedback Agent created:]
[2024-06-22 15:51:12,043: INFO: 3165304408: Cover Letter Editor Agent created:]
[2024-06-22 15:51:12,044: INFO: 3165304408: Job Description Summary Agent created:]
[2024-06-22 15:51:12,055: INFO: 3165304408: Agent graph created:]


In [46]:
from IPython.display import display, Markdown

In [51]:
display(Markdown(cover_letter_draft))

Dear Sandra P.,

I am excited to apply for the Data Architect position at Remobi, where I can utilize my skills and experience to design, create, deploy, and manage data architecture solutions. With a strong background in computational physics and over 3 years of experience working with Python, SQL, and cloud computing, I am confident in my ability to capture and deliver requirements for data visualization dashboards and datahubs.

Throughout my career, I have developed and maintained data pipelines, performed statistical analysis, and developed machine learning models, including a CNN for noise classification. My experience with AWS, Azure, and Docker has equipped me with the knowledge to set up and manage servers, VPC, and storage, as well as implement containerization for data pipelines. Additionally, I have set up CI/CD systems using Jenkins and have experience with Linux, setting up and maintaining on-prem and cloud servers.

My experience with LLM development, including creating Neo4j Graph databases and Pinecone vector databases for retrieval, has given me a strong understanding of data governance and data quality principles. I am well-versed in data security and privacy regulations and possess excellent communication and collaboration skills, which have been essential in my previous roles.

I am a continuous learner, always willing to help, and thrive in fast-paced environments. I am excited about the opportunity to join Remobi and contribute to building the world's greatest community of remote technologists. I am confident that my skills and experience make me a strong fit for this role, and I look forward to discussing my application further.

Thank you for considering my application.

Sincerely,
Alejandro Maza Villalpando

In [52]:
display(Markdown(final_cover_letter))

Dear Sandra P.,

With my background in computational physics and experience in data architecture, I am confident that I can make a valuable contribution to Remobi's mission to build the world's greatest community of remote technologists. I am excited to apply for the Data Architect position, where I can utilize my skills and experience to design, create, deploy, and manage data architecture solutions.

Remobi's commitment to building a community of remote technologists resonates with me, and I am drawn to the company's innovative approach to data architecture. With my experience in developing and maintaining data pipelines, performing statistical analysis, and developing machine learning models, I believe I can make a meaningful impact at Remobi. My knowledge of cloud computing, Docker, and Jenkins has equipped me with the skills to set up and manage servers, VPC, and storage, as well as implement containerization for data pipelines.

I am excited about the opportunity to bring my skills and experience to Remobi and contribute to building the world's greatest community of remote technologists. I am confident that my strong understanding of data governance and data quality principles, as well as my excellent communication and collaboration skills, make me a strong fit for this role. I look forward to discussing my application further and exploring how I can make a meaningful impact at Remobi.

Sincerely,
Alejandro Maza Villalpando